<a href="https://colab.research.google.com/github/winny0217/gpt2/blob/main/gpt2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

In [10]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# 載入模型與 tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

if torch.cuda.is_available():
    model.to("cuda")

def get_top_k_predictions(input_text, top_k=3, max_predict_len=3):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    if torch.cuda.is_available():
        input_ids = input_ids.to("cuda")

    generated_ids = input_ids.clone()

    suggestions = []

    for _ in range(max_predict_len):
        with torch.no_grad():
            outputs = model(generated_ids)
            logits = outputs.logits

        next_token_logits = logits[:, -1, :]  # 最後一個 token 的預測
        probs = torch.softmax(next_token_logits, dim=-1)

        # 選出 top_k 的 token
        top_k_probs, top_k_ids = torch.topk(probs, top_k)

        # 解碼成字詞
        tokens = [tokenizer.decode([token_id]) for token_id in top_k_ids[0]]
        suggestions.append(tokens)

        # 模擬「選擇第一個建議」
        selected_id = top_k_ids[0][0].unsqueeze(0).unsqueeze(0)
        generated_ids = torch.cat((generated_ids, selected_id), dim=1)

    return suggestions

In [11]:
# ==== 示範輸入 ====
input_text = "I want to go to the"

suggestions = get_top_k_predictions(input_text)

print(f"Input text: {input_text}")
for i, options in enumerate(suggestions):
    print(f"Suggestion for word {i+1}: {options}")

Input text: I want to go to the
Suggestion for word 1: [' next', ' gym', ' hospital']
Suggestion for word 2: [' level', ' stage', ' step']
Suggestion for word 3: ['.', ',"', ',']
